# ロジスティック回帰で予測するサンプル

In [1]:
import pandas as pd
import numpy as np

import re

from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
df = pd.read_csv('./datasets/titanic/train.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [3]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,0.363636,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.481622,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,0.000000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,0.000000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,1.000000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,1.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [4]:
test_df = pd.read_csv('./datasets/titanic/test.csv')
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


## 名前の前処理family name, last nameに分割

In [5]:
title_pattern = re.compile('[a-zA-Z]+\.\s')
LAST='LastName'
FAMILY='FamilyName'

def _preprocess_name(df: pd.DataFrame) -> pd.DataFrame:
    name_df = df['Name'].str.split(',', expand=True)
    name_df.columns = [LAST, FAMILY]
    name_df[FAMILY] = name_df[FAMILY].str.replace(title_pattern, '', regex=True).str.strip()
    df = df.drop(columns='Name')
    name_df = name_df.drop(columns='LastName')
    return pd.concat([df, name_df], axis=1)

## pclassの前処理 欠損埋めしてスケーリング

In [6]:
PCLASS= 'Pclass'

def _preprocess_pclass(df: pd.DataFrame) -> pd.DataFrame:
    df[PCLASS] = df[PCLASS].fillna(0)
    
    scaler = MinMaxScaler()
    scaler.fit(df[[PCLASS]])
    
    df[PCLASS] = scaler.transform(df[[PCLASS]])
    return df
    

## Age, Fareの前処理　欠損埋めしてスケーリング

In [7]:
AGE = 'Age'
FARE = 'Fare'
SIBSP = 'SibSp'
PARCH = 'Parch'


def _preprocess_age_and_fare_sibsp_parch(df: pd.DataFrame) -> pd.DataFrame:
    df['Age_isnull'] = df[AGE].isnull().astype(int)
    df['Fare_isnull'] = df[FARE].isnull().astype(int)
    
    mean_age = df[AGE].mean()
    mean_fare = df[FARE].mean()
    
    df[AGE] = df[AGE].fillna(mean_age)
    df[FARE] = df[FARE].fillna(mean_fare)
    df[SIBSP] = df[SIBSP].fillna(0)
    df[PARCH] = df[PARCH].fillna(0)
    
    scaler = StandardScaler()
    scaler.fit(df[[AGE, FARE, SIBSP, PARCH]])
    
    df[[AGE, FARE, SIBSP, PARCH]] = scaler.transform(df[[AGE, FARE, SIBSP, PARCH]])
    return df
    
    

In [8]:
def _preprocess_sex_cabin_embarked(df: pd.DataFrame) -> pd.DataFrame:
    dummy_df = pd.get_dummies(df, drop_first=True, dummy_na=True, columns=['Sex', 'Cabin', 'Embarked'])
    return dummy_df

In [9]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop(columns=['PassengerId', 'Ticket', 'Name'])

    # df = _preprocess_name(df)
    df = _preprocess_pclass(df)
    df = _preprocess_age_and_fare_sibsp_parch(df)
    df = _preprocess_sex_cabin_embarked(df)
    return df

In [10]:
prep_df = preprocess(df)

In [11]:
prep_df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare,Age_isnull,Fare_isnull,Sex_male,Sex_nan,...,Cabin_F E57,Cabin_F G63,Cabin_F2,Cabin_F33,Cabin_F4,Cabin_G6,Cabin_nan,Embarked_Q,Embarked_S,Embarked_nan
count,418.000000,418.000000,4.180000e+02,4.180000e+02,4.180000e+02,4.180000e+02,418.000000,418.000000,418.000000,418.0,...,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.0
mean,0.363636,0.632775,3.187243e-17,-1.593622e-17,-2.974760e-17,-1.806104e-17,0.205742,0.002392,0.636364,0.0,...,0.002392,0.002392,0.002392,0.002392,0.004785,0.002392,0.782297,0.110048,0.645933,0.0
std,0.481622,0.420919,1.001198e+00,1.001198e+00,1.001198e+00,1.001198e+00,0.404727,0.048912,0.481622,0.0,...,0.048912,0.048912,0.048912,0.048912,0.069088,0.048912,0.413179,0.313324,0.478803,0.0
min,0.000000,0.000000,-2.385419e+00,-4.994700e-01,-4.002477e-01,-6.387815e-01,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,0.000000,0.000000,-5.763018e-01,-4.994700e-01,-4.002477e-01,-4.972129e-01,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0
50%,0.000000,1.000000,0.000000e+00,-4.994700e-01,-4.002477e-01,-3.796234e-01,0.000000,0.000000,1.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.0
75%,1.000000,1.000000,4.340463e-01,6.169924e-01,-4.002477e-01,-7.399887e-02,0.000000,0.000000,1.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.0
max,1.000000,1.000000,3.623577e+00,8.432229e+00,8.781044e+00,8.547081e+00,1.000000,1.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

## 25%でhold-outしてバリデーションデータを作成

In [13]:
from sklearn.model_selection import train_test_split

# ① 学習用データとバリデーションデータを用意
df_x = prep_df.drop(columns=['Survived'])
df_y = prep_df['Survived']

train_x, valid_x, train_y, valid_y = train_test_split(df_x, df_y, test_size=0.25, random_state=42)

## 学習

In [48]:
from keras.models import Sequential
from keras.layers import BatchNormalization, Dense, Dropout
from keras import optimizers

model = Sequential([
    Dense(units=train_x.shape[1], activation='relu'), # 入力層は列数分のユニット
    Dropout(rate=0.2),
    BatchNormalization(),
    Dense(units=16, activation='relu'),
    Dropout(rate=0.2),
    Dense(units=1, activation='sigmoid')              # 出力層は1ユニット
])

model.compile(optimizer=optimizers.SGD(learning_rate=0.01), loss='binary_crossentropy', metrics=['binary_crossentropy'])
model.fit(x=train_x, y=train_y, validation_data=(valid_x, valid_y), batch_size=32, epochs=200)

Epoch 1/200
10/10 [==============================] - 1s 19ms/step - loss: 0.8082 - binary_crossentropy: 0.8082 - val_loss: 0.6770 - val_binary_crossentropy: 0.6770
Epoch 2/200
10/10 [==============================] - 0s 4ms/step - loss: 0.7015 - binary_crossentropy: 0.7015 - val_loss: 0.6500 - val_binary_crossentropy: 0.6500
Epoch 3/200
10/10 [==============================] - 0s 4ms/step - loss: 0.5961 - binary_crossentropy: 0.5961 - val_loss: 0.6290 - val_binary_crossentropy: 0.6290
Epoch 4/200
10/10 [==============================] - 0s 4ms/step - loss: 0.5080 - binary_crossentropy: 0.5080 - val_loss: 0.6025 - val_binary_crossentropy: 0.6025
Epoch 5/200
10/10 [==============================] - 0s 4ms/step - loss: 0.4361 - binary_crossentropy: 0.4361 - val_loss: 0.5800 - val_binary_crossentropy: 0.5800
Epoch 6/200
10/10 [==============================] - 0s 4ms/step - loss: 0.4170 - binary_crossentropy: 0.4170 - val_loss: 0.5578 - val_binary_crossentropy: 0.5578
Epoch 7/200
10/10 [==

In [24]:
train_x

,Pclass,Age,SibSp,Parch,Fare,Age_isnull,Fare_isnull,Sex_male,Sex_nan,Cabin_A18,...,Cabin_F E57,Cabin_F G63,Cabin_F2,Cabin_F33,Cabin_F4,Cabin_G6,Cabin_nan,Embarked_Q,Embarked_S,Embarked_nan
132,1.0,0.000000,-0.49947,3.680326,-0.182174,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
60,1.0,-1.051760,-0.49947,-0.400248,-0.497213,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
198,0.5,-0.576302,-0.49947,-0.400248,-0.450521,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
332,1.0,0.000000,-0.49947,-0.400248,-0.509240,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
63,1.0,-0.655545,-0.49947,-0.400248,-0.500275,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,1.0,-0.734788,-0.49947,-0.400248,-0.497213,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
106,1.0,-0.734788,-0.49947,-0.400248,-0.498558,0,0,1,0,0,...,0,0,0,0,0,0,1,1,0,0
270,0.0,1.246287,-0.49947,-0.400248,0.710273,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
348,0.5,-0.497059,-0.49947,-0.400248,-0.396732,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0


## バリデーションデータで評価

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
y_pred_train = model.predict_proba(train_x)
y_pred = model.predict_proba(valid_x)

logloss_train = log_loss(y_true=train_y, y_pred=y_pred_train)
logloss_valid = log_loss(y_true=valid_y, y_pred=y_pred)

In [ ]:
print(f"logloss - train: {logloss_train}")
print(f"logloss - valid: {logloss_valid}")

In [ ]:
import joblib
joblib.dump(model, 'model.joblib')

model = joblib.load('model.joblib')
m2

In [ ]:
prep_test_df = preprocess(test_df)

In [ ]:
model.predict_proba(prep_test_df)